In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\jgaur\Tensorflow_Tut\LSTM\fake-news\train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
# Drop Nan values
df = df.dropna()

In [4]:
# Get the independent fetures

X = df.drop('label', axis=1)

In [5]:
# Get the dependent features

y = df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
import tensorflow as tf

In [8]:
tf.__version__

'2.3.1'

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from  tensorflow.keras.preprocessing.text import one_hot

In [10]:
# Vocabulary size
voc_size = 5000

In [11]:
messages = X.copy()

In [12]:
messages.reset_index(inplace=True)  # bcz we have droped the Nan values

In [13]:
import nltk
import re 
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jgaur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    # print(review[:5])
    review = review.split()
    # print(review[:5])
    review = [word for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
one_hot = [one_hot(word, voc_size) for word in corpus]

In [23]:
sen_length = 25
pad_sequence = pad_sequences(one_hot, padding='pre', maxlen=sen_length)
print(pad_sequence)

[[   0    0    0 ... 4180  498 4163]
 [   0    0    0 ...  683 3960 1313]
 [   0    0    0 ... 3920 4857 3950]
 ...
 [   0    0    0 ... 2928  833 1776]
 [   0    0    0 ...  489 4089 2476]
 [   0    0    0 ... 4747 3494 1277]]


In [24]:
len(pad_sequence)

18285

In [25]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sen_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

# model.compile tell the model what loss and optimization mehtod to use
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
y.shape

(18285,)

In [29]:
import numpy as np 
X_final = np.array(pad_sequence)
y_final = np.array(y)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [31]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 72ms/step - loss: 0.3297 - accuracy: 0.8440 - val_loss: 0.1948 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 9s 49ms/step - loss: 0.1340 - accuracy: 0.9473 - val_loss: 0.1983 - val_accuracy: 0.9171
Epoch 3/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0890 - accuracy: 0.9693 - val_loss: 0.2413 - val_accuracy: 0.9193
Epoch 4/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0541 - accuracy: 0.9828 - val_loss: 0.2742 - val_accuracy: 0.9152
Epoch 5/10
192/192 [==============================] - 13s 70ms/step - loss: 0.0307 - accuracy: 0.9918 - val_loss: 0.3694 - val_accuracy: 0.9089
Epoch 6/10
192/192 [==============================] - 11s 60ms/step - loss: 0.0171 - accuracy: 0.9950 - val_loss: 0.4407 - val_accuracy: 0.8971
Epoch 7/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0103 - accuracy: 0.9967 - val_loss: 0.4791 - val_accuracy: 0

In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sen_length))
model.add(tf.keras.layers.Dropout(0.3))
model.add(LSTM(100))
model.add(tf.keras.layers.Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())